# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
part_one_file = "output_data/cities.csv"
part_one_df = pd.read_csv(part_one_file)
part_one_df

,city,country,latitude,longitude,max_temp,humidity,cloudiness,Wind speed
0,camacupa,AO,-12.02,17.48,16.77,95,100,1.40
1,springbok,ZA,-29.67,17.88,14.00,93,100,3.60
2,mporokoso,ZM,-9.37,30.12,16.41,95,87,1.17
3,oum hadjer,TD,13.30,19.70,19.29,20,1,4.86
4,lichtenburg,ZA,-26.15,26.16,19.20,61,100,5.56
...,...,...,...,...,...,...,...,...
311,luangwa,MZ,-15.62,30.40,21.81,92,100,0.15
312,caala,AO,-12.85,15.56,16.44,95,100,0.91
313,maamba,ZM,-17.36,27.22,21.92,82,100,0.50
314,yagoua,CM,10.34,15.23,18.67,20,0,3.69


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [59]:
# Configure gmaps
gmaps.configure(api_key=g_key)
g_key

#Define locations
locations = part_one_df[["latitude", "longitude"]]
#Define weight
humidity = part_one_df["humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
humidity_heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 1)

# Add layer
fig.add_layer(humidity_heatmap)

# Display figure
fig
plt.savefig('Heatmap_with_bad_markers')

<Figure size 432x288 with 0 Axes>

In [8]:
# Declare weather conditions limits

min_humid = 20
max_humid = 70
min_temp = 18
max_temp = 28
max_wind = 5.0

# Filter Dataframe so all limits are obeyed
vacay_df = part_one_df.loc[(part_one_df["humidity"]>= min_humid), :] 
vacay_df = vacay_df.loc[(vacay_df["humidity"]<= max_humid), :] 
vacay_df = vacay_df.loc[(vacay_df["max_temp"]>= min_temp), :] 
vacay_df = vacay_df.loc[(vacay_df["max_temp"]<= max_temp), :] 
vacay_df = vacay_df.loc[(vacay_df["Wind speed"]<= max_wind), :]

vacay_df

,city,country,latitude,longitude,max_temp,humidity,cloudiness,Wind speed
3,oum hadjer,TD,13.30,19.70,19.29,20,1,4.86
9,gobabis,NaN,-22.45,18.97,20.11,65,100,2.98
10,bara,NG,10.41,10.60,18.28,25,100,3.37
14,birao,CF,10.30,22.78,20.61,28,0,4.95
26,impfondo,CD,1.62,18.06,21.77,43,13,0.34
...,...,...,...,...,...,...,...,...
299,bossangoa,CF,6.50,17.45,19.86,40,0,1.21
303,mobaye,CF,4.33,21.18,21.17,44,46,1.56
304,krugersdorp,ZA,-26.10,27.78,22.78,52,40,3.60
306,sefophe,BW,-22.18,27.97,26.00,65,0,4.63


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = vacay_df.loc[:,["city", "country", "latitude", "longitude"]]
hotel_df.reset_index(drop = True, inplace = True)
hotel_df

,city,country,latitude,longitude
0,oum hadjer,TD,13.30,19.70
1,gobabis,NaN,-22.45,18.97
2,bara,NG,10.41,10.60
3,birao,CF,10.30,22.78
4,impfondo,CD,1.62,18.06
...,...,...,...,...
85,bossangoa,CF,6.50,17.45
86,mobaye,CF,4.33,21.18
87,krugersdorp,ZA,-26.10,27.78
88,sefophe,BW,-22.18,27.97


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "key": g_key,
}

latitude = 19.91
longitude = 73.23
params['location'] = f"{latitude},{longitude}"
params['types'] = 'lodging'
response = requests.get(base_url, params=params).json()
response['results'][0]['name']


'Hotel Shyam Palace'

In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "types": "lodging",
    "key": g_key,
}

hotel_list = []

# use iterrows to iterate through pandas dataframe
for index, row in vacay_df.iterrows():

    # get restaurant type from df
    latitude = row['latitude']
    longitude = row['longitude']

    # add keyword to params dict
    params['location'] = f"{latitude},{longitude}"

    # assemble url and make API request
    try:
        response = requests.get(base_url, params=params).json()
        hotel_list.append(response['results'][0]['name'])
    except (KeyError, IndexError):
        hotel_list.append("unavailable")


hotel_df["Hotel Name"] = hotel_list
hotel_df
   

,city,country,latitude,longitude,Hotel Name
0,oum hadjer,TD,13.30,19.70,unavailable
1,gobabis,NaN,-22.45,18.97,Omara Guesthouse
2,bara,NG,10.41,10.60,unavailable
3,birao,CF,10.30,22.78,unavailable
4,impfondo,CD,1.62,18.06,Préfecture de la Likouala
...,...,...,...,...,...
85,bossangoa,CF,6.50,17.45,LA PERLE DE BOSTON
86,mobaye,CF,4.33,21.18,unavailable
87,krugersdorp,ZA,-26.10,27.78,New Bosi
88,sefophe,BW,-22.18,27.97,unavailable


In [16]:
hotel_trim_df = hotel_df.loc[hotel_df["Hotel Name"]!= "unavailable",:]
hotel_final_df = hotel_trim_df.reset_index()
del hotel_final_df['index']
hotel_final_df

,city,country,latitude,longitude,Hotel Name
0,gobabis,NaN,-22.45,18.97,Omara Guesthouse
1,impfondo,CD,1.62,18.06,Préfecture de la Likouala
2,isiro,CD,2.77,27.62,Nova
3,bumba,CD,2.18,22.47,Hôtel Cape Town
4,buta,CD,2.79,24.73,HOTEL BISO NA BISO
5,bunia,CD,1.56,30.25,Hotel Canadian
6,karasburg,NaN,-28.01,18.75,Avondsrus Guesthouse
7,keetmanshoop,NaN,-26.57,18.15,Suiderlig Hostel
8,okahandja,NaN,-21.98,16.91,Sylvanette Guesthouse
9,kibala,TD,9.11,18.35,Auberge Ndoumbelane


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_final_df.iterrows()]
locations = hotel_final_df[["latitude", "longitude"]]

In [49]:
# Add marker layer ontop of heat map

#for row in hotel_final_df.iterrows():
list = [2,3]
    
mark = gmaps.Marker(list)


# Display Map

fig

Figure(layout=FigureLayout(height='420px'))